In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread

import tensorflow as tf

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, Lambda, Reshape, multiply
from keras.layers import Activation, Dropout, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Layer
from sklearn.model_selection import train_test_split

from keras.optimizers import RMSprop,SGD
from keras.regularizers import l2

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
'''physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)'''

"physical_devices = tf.config.experimental.list_physical_devices('GPU')\nif len(physical_devices) > 0:\n    tf.config.experimental.set_memory_growth(physical_devices[0], True)"

In [4]:
#!unzip /content/AML.zip -d /content/

In [5]:
#!unzip /content/CALL.zip -d /content/

In [6]:
#!pip install imgaug

In [7]:
# Ruta de la carpeta de descargas
# Directorio de descargas
download_dir = os.path.expanduser("AML/AML1")  # Ruta de la carpeta de descargas

# Subdirectorios dentro de la carpeta de descargas
#subdirs = ["M0", "M1", "M2", "M3", "M4"]
subdirs = ["all", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "sanos"]

# Listas para almacenar imágenes y recuentos
images = []
directories = []
dircount = []

# Tamaño deseado de las imágenes redimensionadas
target_height = 150 #108, 200
target_width = 250  #160, 300

print("leyendo imagenes de ",download_dir)

# Recorrer cada subdirectorio
for subdir in subdirs:
    subdir_path = os.path.join(download_dir, subdir)
    images_count = 0

    # Recorrer archivos en el subdirectorio
    for root, _, filenames in os.walk(subdir_path):
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                filepath = os.path.join(root, filename)

                # Leer y redimensionar la imagen
                image = imread(filepath)
                image_resized = resize(image, (target_height, target_width), anti_aliasing=True, preserve_range=True)
                images.append(image_resized)
                #images.append(image)

                images_count += 1

    # Almacenar recuento y directorio
    dircount.append(images_count)
    directories.append(subdir)

# Mostrar estadísticas finales
print('Directorios leídos:', len(directories))
print("Imágenes en cada directorio:", dircount)
print('Suma total de imágenes en subdirectorios:', sum(dircount))

# Convertir la lista de imágenes a un array de numpy
#images_array = np.array(images)
'''for i in range(len(images)):
  images[i] = np.array(images[i], dtype=np.uint8)'''
# Convertir las imágenes a RGB
'''for i in range(len(images)):
  images[i] = images[i][..., :3]
'''
type(images)

leyendo imagenes de  AML/AML1
Directorios leídos: 9
Imágenes en cada directorio: [360, 360, 360, 360, 360, 360, 360, 360, 360]
Suma total de imágenes en subdirectorios: 3240


list

In [8]:
# Directorio que contiene las imágenes
directorio_imagenes = os.path.expanduser("AML/AML1")
directorio_imagenes

'AML/AML1'

# Nueva sección

---



In [9]:
labels = []
indice = 0
for cantidad in dircount:
  for i in range(cantidad):
    labels.append(indice)
  indice = indice + 1
print("Cantidad de etiquetas creadas: ", len(labels))

Cantidad de etiquetas creadas:  3240


In [10]:
AMLI=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    AMLI.append(name[len(name)-1])
    indice=indice+1

0 all
1 M1
2 M2
3 M3
4 M4
5 M5
6 M6
7 M7
8 sanos


In [11]:
type(AMLI)

list

In [12]:
AMLI

['all', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'sanos']

In [13]:
#for img in images:
    #print(img.shape)

In [14]:
y = np.array(labels)
#X = images
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
type(X)

Total number of outputs :  9
Output classes :  [0 1 2 3 4 5 6 7 8]


numpy.ndarray

In [15]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir listas a arrays NumPy
#X_train = np.array(X_train)
#X_test = np.array(X_test)

# Normalizamos como float32 para poder dividirlos entre 0 y 1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Valores entre 0..1
X_train /= 255
X_test /= 255

print('Training data shape : ', X_train.shape, y_train.shape)
print('Testing data shape : ', X_test.shape, y_test.shape)

img_size = X[0].shape
print(img_size)

Training data shape :  (2592, 150, 250, 3) (2592,)
Testing data shape :  (648, 150, 250, 3) (648,)
(150, 250, 3)


In [16]:
from keras.utils import to_categorical

# Codificar las etiquetas
y_train_encoded = to_categorical(y_train, num_classes=nClasses)
y_test_encoded = to_categorical(y_test, num_classes=nClasses)

# Modelos


---



In [17]:
# Modelo ResNet
resnet_model = ResNet50(weights=None, input_shape=img_size, classes=nClasses)

In [18]:
epochs = 20
batch_size = 32
train_size =2592

initial_learning_rate = 0.001
final_learning_rate = 0.00001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/epochs)
steps_per_epoch = int(train_size/batch_size)

lr_schedule2 = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)
# Define optimizer using the learning rate schedule
optimizer2 = RMSprop(learning_rate=lr_schedule2)

In [19]:
resnet_model.compile(optimizer=optimizer2, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
resnet_model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test_encoded))

Epoch 1/20
81/81 [==============================] - 197s 2s/step - loss: 2.7749 - accuracy: 0.2346 - val_loss: 46.9418 - val_accuracy: 0.0910
Epoch 2/20
81/81 [==============================] - 186s 2s/step - loss: 1.7902 - accuracy: 0.3152 - val_loss: 2.9797 - val_accuracy: 0.1034
Epoch 3/20
81/81 [==============================] - 186s 2s/step - loss: 1.6222 - accuracy: 0.3900 - val_loss: 2.2260 - val_accuracy: 0.1157
Epoch 4/20
81/81 [==============================] - 163s 2s/step - loss: 1.4322 - accuracy: 0.4468 - val_loss: 2.1624 - val_accuracy: 0.3194
Epoch 5/20
81/81 [==============================] - 163s 2s/step - loss: 1.2844 - accuracy: 0.5120 - val_loss: 1.8291 - val_accuracy: 0.4074
Epoch 6/20
81/81 [==============================] - 154s 2s/step - loss: 1.1293 - accuracy: 0.5737 - val_loss: 1.7887 - val_accuracy: 0.4691
Epoch 7/20
81/81 [==============================] - 161s 2s/step - loss: 0.9934 - accuracy: 0.6204 - val_loss: 1.4024 - val_accuracy: 0.4923
Epoch 8/20
8

In [ ]:
resnet_model.save('resnet_model.h5')